#### This notebook shows how to read the fastMRI dataset and apply some simple transformations to the data.

#### This notebook does not include the raw data files. In order to run the notebook, you need to download them from https://fastmri.med.nyu.edu/

In [1]:
%matplotlib inline
import os
import h5py
import numpy as np
from matplotlib import pyplot as plt

The fastMRI dataset is distributed as a set of HDF5 files and can be read with the h5py package. Here, we show how to open a file from the multi-coil dataset. Each file corresponds to one MRI scan and contains the k-space data, ground truth and some meta data related to the scan.

In [2]:
base_path = r"W:\fastMRI_brain\multicoil_train"
file_name = "file_brain_AXT2_210_6001944.h5"
full_path = os.path.join(base_path, file_name)
hf = h5py.File(full_path)

In [3]:
print('Keys:', list(hf.keys()))
print('Attrs:', dict(hf.attrs))

Keys: ['ismrmrd_header', 'kspace', 'reconstruction_rss']
Attrs: {'acquisition': 'AXT2', 'max': 0.0007081719243083575, 'norm': 0.16223164878819385, 'patient_id': '9f1bb52e08717d5f753d9d076f101a7fd971d3f906c3ef8c318caad206891389'}


In multi-coil MRIs, k-space has the following shape:
(number of slices, number of coils, height, width)

For single-coil MRIs, k-space has the following shape:
(number of slices, height, width)

MRIs are acquired as 3D volumes, the first dimension is the number of 2D slices.

In [4]:
volume_kspace = hf['kspace'][()]
print(volume_kspace.dtype)
print(volume_kspace.shape)

complex64
(16, 20, 768, 396)


In [5]:
slice_kspace = volume_kspace[6] # Choosing the 20-th slice of this volume

Let's see what the absolute value of k-space looks like:

The fastMRI repo contains some utlity functions to convert k-space into image space. These functions work on PyTorch Tensors. The to_tensor function can convert Numpy arrays to PyTorch Tensors.

In [6]:
import fastmri
from fastmri.data import transforms as T

C:\Users\AIWM_PC\K-space\fastmri\__init__.py:16: UserWarning: Could not retrieve fastmri version!
  warnings.warn("Could not retrieve fastmri version!")


In [7]:
slice_kspace2 = T.to_tensor(slice_kspace)      # Convert from numpy array to pytorch tensor
slice_image = fastmri.ifft2c(slice_kspace2)           # Apply Inverse Fourier Transform to get the complex image
slice_image_abs = fastmri.complex_abs(slice_image)   # Compute absolute value to get a real image

As we can see, each coil in a multi-coil MRI scan focusses on a different region of the image. These coils can be combined into the full image using the Root-Sum-of-Squares (RSS) transform.

In [ ]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from tqdm import tqdm

# Assume slice_kspace is the given data (shape: [coil, height, width])
# Example: slice_kspace = np.random.randn(n_coils, height, width) + 1j * np.random.randn(n_coils, height, width)

# Extract data information
n_coils, height, width = slice_kspace.shape
n_pixels = height * width  # Total number of pixels

# Reshape data to (n_pixels, n_coils)
A = slice_kspace.reshape(n_coils, n_pixels).T  # Shape: (n_pixels, n_coils)

# Compute ground truth (using RSS method)
b = np.sqrt(np.sum(np.abs(A)**2, axis=1))  # Shape: (n_pixels,)

# Define cost function
def cost_function(x, A, b):
    """
    Cost function: minimize the L2 norm of sqrt(|Ax|) - sqrt(|b|)
    """
    x = x.view(np.complex128)  # Convert real vector to complex
    Ax = np.dot(A, x)
    error = np.sqrt(np.abs(Ax)) - np.sqrt(np.abs(b))
    return np.linalg.norm(error)**2

# Set initial values
x0_real = np.mean(slice_kspace.real, axis=(1, 2))  # Mean of real part for each coil
x0_imag = np.mean(slice_kspace.imag, axis=(1, 2))  # Mean of imaginary part for each coil
x0 = np.hstack((x0_real, x0_imag))  # Combine real and imaginary parts to create initial values

# Use TQDM to display progress
tqdm_bar = tqdm(total=500, desc="Optimization Progress", unit="iteration")  # Estimate 500 iterations in total

def tqdm_callback(xk):
    """
    Callback function called during optimization
    """
    tqdm_bar.update(1)  # Update progress by 1 step
    current_loss = cost_function(xk, A, b)  # Compute current loss
    tqdm_bar.set_postfix(loss=f"{current_loss:.4e}")

# Run optimization
result = minimize(
    fun=cost_function,
    x0=x0,
    args=(A, b),
    method='L-BFGS-B',
    options={'disp': False, 'maxiter': 500},
    callback=tqdm_callback  # Pass progress callback
)

# Close the progress bar
tqdm_bar.close()

# Recover optimized x
x_optimized = result.x.view(np.complex128)

# Generate single-coil data
single_coil_data = np.dot(A, x_optimized)  # Shape: (n_pixels,)
single_coil_image = single_coil_data.reshape(height, width)  # Restore original image size

# Check results
print("Optimized coefficients (x):", x_optimized)
print("Single-coil image size:", single_coil_image.shape)

# Visualize single-coil image
plt.imshow(np.abs(single_coil_image), cmap='gray')
plt.title("Single Coil Image")
plt.show()


Optimization Progress:   0%|                                                            | 0/500 [00:00<?, ?iteration/s]

In [ ]:
slice_kspace2 = T.to_tensor(single_coil_image)      # Convert from numpy array to pytorch tensor
slice_image = fastmri.ifft2c(slice_kspace2)           # Apply Inverse Fourier Transform to get the complex image
slice_image_abs = fastmri.complex_abs(slice_image)   # Compute absolute value to get a real image

So far, we have been looking at fully-sampled data. We can simulate under-sampled data by creating a mask and applying it to k-space.

In [ ]:
fig = plt.figure()
plt.imshow(slice_image_abs, cmap='gray')

In [ ]:
from fastmri.data.subsample import RandomMaskFunc
mask_func = RandomMaskFunc(center_fractions=[0.04], accelerations=[12])  # Create the mask function object

In [ ]:
masked_kspace, mask, _ = T.apply_mask(slice_kspace2, mask_func)   # Apply the mask to k-space

In [ ]:
import torch
import matplotlib.pyplot as plt

# Remove singleton dimensions
mask_reshaped = mask.squeeze()  # Remove unnecessary dimensions
print("Mask Shape after Squeeze:", mask_reshaped.shape)

mask_2d = mask_reshaped.repeat(396, 1).numpy()  # Repeat along a new axis to make it 2D
plt.figure(figsize=(10, 5))
plt.imshow(mask_2d, cmap="gray")
plt.title("Visualization of Mask (2D)")
plt.show()

Let's see what the subsampled image looks like:

In [ ]:
slice_image = fastmri.ifft2c(masked_kspace)           # Apply Inverse Fourier Transform to get the complex image
slice_image_abs = fastmri.complex_abs(slice_image)   # Compute absolute value to get a real image

In [ ]:
plt.imshow(np.abs(slice_image_abs.numpy()), cmap='gray')